In [1]:
import os
import torch
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

from trl import SFTTrainer, SFTConfig

from src.utils.config_loader import load_config
from src.utils.model_utils import get_finetuned_model, generate_response
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Set up file paths

In [2]:
config = load_config()

In [3]:
train_dataset_path = config.get("train_dataset_path")
test_dataset_path = config.get("test_dataset_path")
dev_dataset_path = config.get("dev_dataset_path")

In [4]:
checkpoint_path = config.get("checkpoint_path")

### Load datasets and system prompt:

In [5]:
raw_train = load_dataset("json", data_files=train_dataset_path, download_mode="force_redownload")["train"]
raw_test = load_dataset("json", data_files=test_dataset_path, download_mode="force_redownload")["train"]
raw_dev = load_dataset("json", data_files=dev_dataset_path, download_mode="force_redownload")["train"]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

### Load model and tokenizer
We need to load it before processing the data, as we are going to use the tokenizer to format the data.

In [6]:
model_path = config.get("model")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model = prepare_model_for_kbit_training(model)

### Change the dataset format to chat-like text

In [8]:
system_prompt = config.get("system_prompt")

Function below may need to get adjusted based on the model you are using.

In [9]:
def apply_chat_template(sample, tokenizer, include_response=True):
    # Combine the fields into a structured chat format
    message = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": sample["user"]}
    ]
    if include_response:
        message.append({"role": "assistant", "content": sample["assistant"]})

    # Use the tokenizer's chat template to create formatted text
    message = tokenizer.apply_chat_template(
        message, tokenize=False, add_generation_prompt=False
    )
    return tokenizer(message, padding=True, truncation=True)

In [10]:
# Apply processing to each dataset
processed_train = raw_train.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
)
processed_test = raw_test.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
)
processed_dev = raw_dev.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
)

Map:   0%|          | 0/2332 [00:00<?, ? examples/s]

Map:   0%|          | 0/2422 [00:00<?, ? examples/s]

Map:   0%|          | 0/2341 [00:00<?, ? examples/s]

### Train

In [11]:
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        print(name)

model.layers.0.self_attn.o_proj
model.layers.0.self_attn.qkv_proj
model.layers.0.mlp.gate_up_proj
model.layers.0.mlp.down_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.qkv_proj
model.layers.1.mlp.gate_up_proj
model.layers.1.mlp.down_proj
model.layers.2.self_attn.o_proj
model.layers.2.self_attn.qkv_proj
model.layers.2.mlp.gate_up_proj
model.layers.2.mlp.down_proj
model.layers.3.self_attn.o_proj
model.layers.3.self_attn.qkv_proj
model.layers.3.mlp.gate_up_proj
model.layers.3.mlp.down_proj
model.layers.4.self_attn.o_proj
model.layers.4.self_attn.qkv_proj
model.layers.4.mlp.gate_up_proj
model.layers.4.mlp.down_proj
model.layers.5.self_attn.o_proj
model.layers.5.self_attn.qkv_proj
model.layers.5.mlp.gate_up_proj
model.layers.5.mlp.down_proj
model.layers.6.self_attn.o_proj
model.layers.6.self_attn.qkv_proj
model.layers.6.mlp.gate_up_proj
model.layers.6.mlp.down_proj
model.layers.7.self_attn.o_proj
model.layers.7.self_attn.qkv_proj
model.layers.7.mlp.gate_up_proj
model.layers.

We pass in the best parameters we found in the sweep.

In [12]:
peft_config = LoraConfig(
    r=4,
    lora_alpha=64,
    target_modules=['gate_up_proj', 'base_layer', 'down_proj', 'qkv_proj', 'o_proj'],
    lora_dropout=0.3005,
    bias="none",
    task_type="CAUSAL_LM"
)

In [13]:
trainer = SFTTrainer(
    model=model,
    args=SFTConfig(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=2,
        learning_rate=0.0007643,
        max_seq_length=4,
        bf16=True,
        optim="adamw_8bit",
        lr_scheduler_type="cosine",
        warmup_ratio=0.2344,
        logging_steps=10,
        save_strategy="epoch",
        output_dir=checkpoint_path,
        eval_accumulation_steps=50,
    ),
    train_dataset=processed_train,#processed_long_train,
    eval_dataset=processed_dev,
    # eval_steps=200,
    peft_config=peft_config,
    tokenizer=tokenizer
)
trainer.train()

/home/martinh2k3/anaconda3/envs/bp/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


[2025-04-26 00:32:46,925] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/martinh2k3/anaconda3/envs/bp/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/martinh2k3/anaconda3/envs/bp/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/martinh2k3/anaconda3/envs/bp/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/martinh2k3/anaconda3/envs/bp/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/martinh2k3/anaconda3/envs/bp/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/martinh2k3/anaconda3/envs/bp/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/martinh2k3/ana

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/martinh2k3/anaconda3/envs/bp/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
You are not running the flash-attention implementation, expect numerical differences.


Step,Training Loss
10,1.566400
20,0.691700
30,0.467500
40,0.459600
50,0.468400
60,0.446800
70,0.460300
80,0.460100
90,0.455700
100,0.464300


/home/martinh2k3/anaconda3/envs/bp/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=290, training_loss=0.45034664910415123, metrics={'train_runtime': 1322.1645, 'train_samples_per_second': 3.528, 'train_steps_per_second': 0.219, 'total_flos': 3.86554692409344e+16, 'train_loss': 0.45034664910415123, 'epoch': 1.9897084048027445})

In [14]:
trainer.save_model(config.get("model_dir_path") + "/phi3mini")

In [21]:
torch.cuda.empty_cache()

In [14]:
trainer.evaluate()

{'eval_loss': 0.5289265513420105,
 'eval_runtime': 183.3295,
 'eval_samples_per_second': 12.769,
 'eval_steps_per_second': 1.598,
 'epoch': 2.490566037735849}


{'eval_loss': 0.4607459306716919,
 'eval_runtime': 308.9566,
 'eval_samples_per_second': 7.577,
 'eval_steps_per_second': 0.948,
 'epoch': 3.979416809605489}